# Tools with MCP ⏰
# MCP를 활용한 도구 ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

MCP(Model Context Protocol)는 AI 에이전트를 외부 도구 및 데이터 소스에 연결하는 표준화된 방법을 제공합니다. `langchain-mcp-adapters`를 사용하여 MCP 서버에 연결해 봅시다.

## Setup
## 설정

Load and/or check for needed environmental variables

필요한 환경 변수를 로드하거나 확인합니다

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Create an agent with the MCP-provided time tools.

MCP에서 제공하는 시간 도구를 사용하여 에이전트를 생성합니다.

In [ ]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model="openai:gpt-5-nano",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

샌프란시스코의 현재 시간을 물어봅니다.

In [ ]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()